In [43]:
from typing import List
from functools import partial

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from hw1.hw1 import precision_at_k, ap_k, ndcg

## load&split

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0


In [3]:
users, items, interactions, max_week = data.user_id.nunique(), data.item_id.nunique(), data.shape[0], data['week_no'].nunique()

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)
print('# max week: ', max_week)

# users:  2499
# items:  89051
# interactions:  2396804
# max week:  95


In [4]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [5]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


In [6]:
# train/test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

## prepare result frame

In [7]:
# prepare result DataFrame
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [8]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### prepare most popular top-K and pivot matrix

In [9]:
# Топ популярных товаров
def k_popular(source, k=5000):
    """ Топ K наиболее продаваемых товаров """
    popular = source.groupby('item_id')['quantity'].sum().reset_index()
    return popular.sort_values('quantity', ascending=False).head(k).item_id.tolist()

In [10]:
top_5000 = k_popular(data_train, k=5000)

In [11]:
# флаг, что товар не из топа
top_train = data_train.copy()
top_train.loc[~top_train['item_id'].isin(top_5000), 'item_id'] = -1

top_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [12]:
user_item_matrix = pd.pivot_table(top_train,
                                  index='user_id',
                                  columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                 )

# user_item_matrix[-1] = 0        # эти товары не в топе. Это выключит фиктивное предсказание
user_item_matrix[user_item_matrix > 0] = 1
user_item_matrix = user_item_matrix.astype(float)

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [13]:
user_item_matrix.head()

item_id,-1,202291,397896,420647,480014,545926,707683,731106,818980,819063,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# remap dictionaries
id_to_itemid = dict(enumerate(user_item_matrix.columns.values))
id_to_userid = dict(enumerate(user_item_matrix.index.values))

itemid_to_id = {v: k for k, v in id_to_itemid.items()}
userid_to_id = {v: k for k, v in id_to_userid.items()}

## 1. Baselines

### 1.1 Random recommendation

In [15]:
def random_recommendation(_items, n=5):
    """Случайные рекоммендации"""
    _items = np.array(_items)
    _recs = np.random.default_rng().choice(items, size=n, replace=False)
    return _recs.tolist()

In [16]:
%%time
items = data_train.item_id.unique()
result['random'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(3)

CPU times: user 215 ms, sys: 4.44 ms, total: 219 ms
Wall time: 218 ms


,user_id,actual,random
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1035497, 1968985, 1120955, 13210073, 66479]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[984680, 13382151, 878837, 845090, 957866]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5996550, 1333373, 958945, 1960562, 13133787]"


### 1.2 Popularity-based recommendation

In [17]:
def popularity_recommendation(_data, n=5):
    """Топ-n популярных товаров"""
    popular = _data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    _recs = popular.head(n).item_id
    return _recs.tolist()

In [18]:
%%time
# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)
result['most_popular'] = result['user_id'].apply(lambda x: popular_recs)
result.head(3)

CPU times: user 91.9 ms, sys: 0 ns, total: 91.9 ms
Wall time: 90.5 ms


,user_id,actual,random,most_popular
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1035497, 1968985, 1120955, 13210073, 66479]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[984680, 13382151, 878837, 845090, 957866]","[6534178, 6533889, 1029743, 6534166, 1082185]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5996550, 1333373, 958945, 1960562, 13133787]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### 1.3 Weighted random recommender

In [19]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекомендации

    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    _items = items_weights['item_id'].astype(int)
    _weights = items_weights['weight'].astype(float)
    _recs = np.random.default_rng().choice(_items, size=n, replace=False, p=_weights)
    return _recs.tolist()

In [20]:
# Товары можно ранжировать не только по суммарной стоимости, но еще и по количеству проданных единиц
# т.о. в топ не попадут очень дорогие редко продаваемые товары
# с другой стороны, попадут дешевые товары, которые покупаются в больших количествах
# чтобы более-менее уравновесить эти параметры, их можно масштабировать

def calc_weights(source: pd.DataFrame, vec_fields: List[str], beta: List[float]):
    """ Расчет весов
    :param source - исходные данные
    :param vec_fields - признаки, используемые в измерении веса
    :param beta - множитель для изменения значимости признака в оценке веса
    """
    grouped_goods = source.groupby('item_id')[vec_fields].sum()
    beta = np.array(beta)

    # mean scale
    scaled_grouped_goods = ((grouped_goods - grouped_goods.mean()) / grouped_goods.std()) * beta
    # min-max scale
    # scaled_grouped_goods = (grouped_goods - grouped_goods.min()) / (grouped_goods.max() - grouped_goods.min())
    # sum scale
    # scaled_grouped_goods = grouped_goods / grouped_goods.sum()

    # в качестве веса товара берем нормализованную норму вектора товара
    vec_norma = np.linalg.norm(scaled_grouped_goods, ord=2, axis=1)
    grouped_goods['weight'] = vec_norma / vec_norma.sum()

    return grouped_goods.sort_values('weight', ascending=False).reset_index()

In [21]:
%%time
weighted_goods = calc_weights(data_train, vec_fields = ['quantity', 'sales_value'], beta=[1., 1.])
result['weighted_random'] = result['user_id'].apply(lambda x: weighted_random_recommendation(weighted_goods, n=5))
result.head(3)

CPU times: user 2.85 s, sys: 15.4 ms, total: 2.86 s
Wall time: 2.87 s


,user_id,actual,random,most_popular,weighted_random
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1035497, 1968985, 1120955, 13210073, 66479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12384897, 66750, 1122350, 15596962, 9193875]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[984680, 13382151, 878837, 845090, 957866]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1058997, 10182558, 1028094, 821279, 859075]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5996550, 1333373, 958945, 1960562, 13133787]","[6534178, 6533889, 1029743, 6534166, 1082185]","[871105, 5584645, 1008288, 12523835, 1136222]"


### 1.4 Random from top-K

In [22]:
%%time
result['random_from_top'] = result['user_id'].apply(lambda x: random_recommendation(top_5000, n=5))
result.head(3)

CPU times: user 693 ms, sys: 7.95 ms, total: 701 ms
Wall time: 701 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1035497, 1968985, 1120955, 13210073, 66479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12384897, 66750, 1122350, 15596962, 9193875]","[7442953, 5996611, 10119550, 5996261, 10150076]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[984680, 13382151, 878837, 845090, 957866]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1058997, 10182558, 1028094, 821279, 859075]","[1079171, 995896, 1962521, 1069614, 392610]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5996550, 1333373, 958945, 1960562, 13133787]","[6534178, 6533889, 1029743, 6534166, 1082185]","[871105, 5584645, 1008288, 12523835, 1136222]","[981038, 1021546, 908361, 827366, 941396]"


### 1.5 Weighted random from top-K

In [23]:
%%time
top_weighted_goods = calc_weights(data_train[data_train['item_id'].isin(top_5000)], vec_fields = ['quantity', 'sales_value'], beta=[1., 1.])
result['weighted_random_from_top'] = result['user_id'].apply(lambda x: weighted_random_recommendation(top_weighted_goods, n=5))
result.head(3)

CPU times: user 878 ms, sys: 20 ms, total: 898 ms
Wall time: 896 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1035497, 1968985, 1120955, 13210073, 66479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12384897, 66750, 1122350, 15596962, 9193875]","[7442953, 5996611, 10119550, 5996261, 10150076]","[883003, 995785, 12262830, 1117219, 828331]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[984680, 13382151, 878837, 845090, 957866]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1058997, 10182558, 1028094, 821279, 859075]","[1079171, 995896, 1962521, 1069614, 392610]","[1004906, 995242, 981760, 970119, 965766]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5996550, 1333373, 958945, 1960562, 13133787]","[6534178, 6533889, 1029743, 6534166, 1082185]","[871105, 5584645, 1008288, 12523835, 1136222]","[981038, 1021546, 908361, 827366, 941396]","[13671759, 1059527, 6533889, 1134210, 13512965]"


## 2. Детерминированные алгоритмы item-item

### подготовка функции-рекомендателя

In [24]:
recommender_params = {
    'user_items': sparse_user_item,
    'N': 5, # кол-во рекомендаций
    'filter_already_liked_items': False,
    'filter_items': [itemid_to_id[-1]],
    # 'recalculate_user': False       # из сорсов имплисита: recalculate_user is ignored because this is not a model based algorithm
}

def recommender(user_id, mdl, params):
    rec_score = mdl.recommend(userid_to_id[user_id], **params)
    return [id_to_itemid[rec[0]] for rec in rec_score]


### 2.1 Item-Item Recommender / ItemKNN

In [25]:
%%time
model = ItemItemRecommender(K=5, num_threads=0) # K - кол-во ближайших соседей
model.fit(sparse_user_item.T, show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.8 s, sys: 6.49 ms, total: 1.81 s
Wall time: 330 ms


In [26]:
%%time
result['item_knn'] = result['user_id'].apply(partial(recommender, mdl=model, params=recommender_params))
result.head(3)

CPU times: user 104 ms, sys: 1.43 ms, total: 106 ms
Wall time: 77.7 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1035497, 1968985, 1120955, 13210073, 66479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12384897, 66750, 1122350, 15596962, 9193875]","[7442953, 5996611, 10119550, 5996261, 10150076]","[883003, 995785, 12262830, 1117219, 828331]","[1082185, 981760, 1127831, 995242, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[984680, 13382151, 878837, 845090, 957866]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1058997, 10182558, 1028094, 821279, 859075]","[1079171, 995896, 1962521, 1069614, 392610]","[1004906, 995242, 981760, 970119, 965766]","[1082185, 981760, 1098066, 995242, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5996550, 1333373, 958945, 1960562, 13133787]","[6534178, 6533889, 1029743, 6534166, 1082185]","[871105, 5584645, 1008288, 12523835, 1136222]","[981038, 1021546, 908361, 827366, 941396]","[13671759, 1059527, 6533889, 1134210, 13512965]","[1082185, 981760, 1127831, 995242, 1098066]"


### 2.2 Косинусное сходство и CosineRecommender

In [27]:
%%time
model = CosineRecommender(K=5, num_threads=0)
model.fit(sparse_user_item.T, show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.76 s, sys: 13.9 ms, total: 1.77 s
Wall time: 344 ms


In [28]:
%%time
result['cosine'] = result['user_id'].apply(partial(recommender, mdl=model, params=recommender_params))
result.head(3)

CPU times: user 96 ms, sys: 103 µs, total: 96.1 ms
Wall time: 91.1 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1035497, 1968985, 1120955, 13210073, 66479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12384897, 66750, 1122350, 15596962, 9193875]","[7442953, 5996611, 10119550, 5996261, 10150076]","[883003, 995785, 12262830, 1117219, 828331]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[984680, 13382151, 878837, 845090, 957866]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1058997, 10182558, 1028094, 821279, 859075]","[1079171, 995896, 1962521, 1069614, 392610]","[1004906, 995242, 981760, 970119, 965766]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5996550, 1333373, 958945, 1960562, 13133787]","[6534178, 6533889, 1029743, 6534166, 1082185]","[871105, 5584645, 1008288, 12523835, 1136222]","[981038, 1021546, 908361, 827366, 941396]","[13671759, 1059527, 6533889, 1134210, 13512965]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]"


### 2.3 TF-IDF взвешивание и TFIDFRecommender

In [29]:
%%time
model = TFIDFRecommender(K=5, num_threads=0)
model.fit(sparse_user_item.T, show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.83 s, sys: 35.4 ms, total: 1.86 s
Wall time: 340 ms


In [30]:
%%time
result['tfidf'] = result['user_id'].apply(partial(recommender, mdl=model, params=recommender_params))
result.head(3)

CPU times: user 110 ms, sys: 403 µs, total: 110 ms
Wall time: 97.9 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1035497, 1968985, 1120955, 13210073, 66479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12384897, 66750, 1122350, 15596962, 9193875]","[7442953, 5996611, 10119550, 5996261, 10150076]","[883003, 995785, 12262830, 1117219, 828331]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[984680, 13382151, 878837, 845090, 957866]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1058997, 10182558, 1028094, 821279, 859075]","[1079171, 995896, 1962521, 1069614, 392610]","[1004906, 995242, 981760, 970119, 965766]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5996550, 1333373, 958945, 1960562, 13133787]","[6534178, 6533889, 1029743, 6534166, 1082185]","[871105, 5584645, 1008288, 12523835, 1136222]","[981038, 1021546, 908361, 827366, 941396]","[13671759, 1059527, 6533889, 1134210, 13512965]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 878996, 961554]"


### 2.4 Трюк - по истории конкретного пользователя (K=1)

In [31]:
%%time
model = ItemItemRecommender(K=1, num_threads=0)
model.fit(sparse_user_item.T, show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.76 s, sys: 6.16 ms, total: 1.76 s
Wall time: 298 ms


In [32]:
%%time
result['own_purchases'] = result['user_id'].apply(partial(recommender, mdl=model, params=recommender_params))
result.head(3)

CPU times: user 78.6 ms, sys: 284 µs, total: 78.9 ms
Wall time: 65 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1035497, 1968985, 1120955, 13210073, 66479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12384897, 66750, 1122350, 15596962, 9193875]","[7442953, 5996611, 10119550, 5996261, 10150076]","[883003, 995785, 12262830, 1117219, 828331]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[984680, 13382151, 878837, 845090, 957866]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1058997, 10182558, 1028094, 821279, 859075]","[1079171, 995896, 1962521, 1069614, 392610]","[1004906, 995242, 981760, 970119, 965766]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 826249, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5996550, 1333373, 958945, 1960562, 13133787]","[6534178, 6533889, 1029743, 6534166, 1082185]","[871105, 5584645, 1008288, 12523835, 1136222]","[981038, 1021546, 908361, 827366, 941396]","[13671759, 1059527, 6533889, 1134210, 13512965]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 878996, 961554]","[1082185, 981760, 995242, 1029743, 840361]"


### 2.5 BM25 recommender

In [33]:
%%time
model = BM25Recommender(K=5, num_threads=0)
model.fit(sparse_user_item.T, show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.77 s, sys: 7.87 ms, total: 1.78 s
Wall time: 300 ms


In [34]:
%%time
result['bm25'] = result['user_id'].apply(partial(recommender, mdl=model, params=recommender_params))
result.head(3)

CPU times: user 93.3 ms, sys: 299 µs, total: 93.6 ms
Wall time: 89.5 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn,cosine,tfidf,own_purchases,bm25
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1035497, 1968985, 1120955, 13210073, 66479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12384897, 66750, 1122350, 15596962, 9193875]","[7442953, 5996611, 10119550, 5996261, 10150076]","[883003, 995785, 12262830, 1117219, 828331]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 995242, 1029743, 840361, 904360]","[872137, 901062, 986947, 1098248, 995785]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[984680, 13382151, 878837, 845090, 957866]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1058997, 10182558, 1028094, 821279, 859075]","[1079171, 995896, 1962521, 1069614, 392610]","[1004906, 995242, 981760, 970119, 965766]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 826249, 1127831]","[1044078, 883404, 913202, 833723, 9526563]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5996550, 1333373, 958945, 1960562, 13133787]","[6534178, 6533889, 1029743, 6534166, 1082185]","[871105, 5584645, 1008288, 12523835, 1136222]","[981038, 1021546, 908361, 827366, 941396]","[13671759, 1059527, 6533889, 1134210, 13512965]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 878996, 961554]","[1082185, 981760, 995242, 1029743, 840361]","[951412, 1041259, 901062, 1082185, 1023720]"


## 3. Измерение качества

In [35]:
result.to_csv('predictions.csv', index=False)
result.head()

,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn,cosine,tfidf,own_purchases,bm25
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1035497, 1968985, 1120955, 13210073, 66479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12384897, 66750, 1122350, 15596962, 9193875]","[7442953, 5996611, 10119550, 5996261, 10150076]","[883003, 995785, 12262830, 1117219, 828331]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 995242, 1029743, 840361, 904360]","[872137, 901062, 986947, 1098248, 995785]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[984680, 13382151, 878837, 845090, 957866]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1058997, 10182558, 1028094, 821279, 859075]","[1079171, 995896, 1962521, 1069614, 392610]","[1004906, 995242, 981760, 970119, 965766]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 826249, 1127831]","[1044078, 883404, 913202, 833723, 9526563]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5996550, 1333373, 958945, 1960562, 13133787]","[6534178, 6533889, 1029743, 6534166, 1082185]","[871105, 5584645, 1008288, 12523835, 1136222]","[981038, 1021546, 908361, 827366, 941396]","[13671759, 1059527, 6533889, 1134210, 13512965]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 878996, 961554]","[1082185, 981760, 995242, 1029743, 840361]","[951412, 1041259, 901062, 1082185, 1023720]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5995820, 9884714, 889689, 888104, 977886]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1079913, 1127716, 866063, 928049, 913785]","[13115812, 1058021, 13213136, 840244, 1745068]","[9296965, 6534178, 881154, 6534166, 1100728]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 961554, 1098066]","[1082185, 981760, 1127831, 961554, 826249]","[1082185, 995242, 1029743, 826249, 1127831]","[7024990, 968774, 1013321, 987724, 879635]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[13008402, 7166861, 29977, 909695, 916770]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12352145, 6534178, 5707620, 9884698, 1033234]","[13159169, 5570486, 5572213, 833307, 835994]","[970747, 6534178, 9878513, 999971, 907631]","[1082185, 981760, 1127831, 1098066, 995242]","[1082185, 981760, 1098066, 826249, 1127831]","[1082185, 981760, 1098066, 826249, 1127831]","[1082185, 981760, 995242, 1029743, 840361]","[1044078, 870780, 1036432, 883404, 905582]"


Для некоторых юзеров нет предсказаний:

In [36]:
zero_pred = [item.user_id for _, item in result.iterrows() if (item.str.len() == 0).any()]
result[result['user_id'].isin(zero_pred)]

,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn,cosine,tfidf,own_purchases,bm25
511,650,"[1473658, 1582467, 1499569, 1678776]","[9879689, 999801, 2456888, 2384597, 12456325]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 826060, 3224992, 6533889, 987218]","[912463, 2017356, 831519, 5582466, 837523]","[878996, 1042907, 12262978, 901976, 893018]","[1082185, 981760, 995242, 1029743]","[1082185, 981760, 995242, 1029743]","[1082185, 981760, 995242, 1029743]",[],"[1082185, 1029743, 995242, 6534178]"
581,729,"[1846901, 1991055, 1997708, 2043643, 9393631, ...","[5581664, 10355470, 9812016, 14112117, 503147]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098844, 15716041, 958594, 831213, 951590]","[15483692, 9858918, 5979533, 1373853, 9297199]","[5587656, 825494, 870211, 940726, 1122358]","[1082185, 981760, 995242, 1029743]","[1082185, 981760, 995242, 1029743]","[1082185, 981760, 995242, 1029743]",[],"[1082185, 1029743, 995242, 6534178]"
770,954,"[1469989, 9268840, 12518029, 12670072, 1267064...","[1057283, 5725201, 15632245, 16829385, 12781490]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1213444, 13776458, 1271789, 903479, 1433927]","[920667, 16733402, 366802, 259687, 1036642]","[999270, 953134, 1130944, 1005186, 1071939]","[1082185, 981760, 995242, 1029743]","[1082185, 981760, 995242, 1029743]","[1082185, 981760, 995242, 1029743]",[],"[1082185, 1029743, 995242, 6534178]"
1625,1987,"[1249892, 1404999, 5664526, 6943103, 17109744,...","[2328229, 854357, 857574, 1403730, 570861]","[6534178, 6533889, 1029743, 6534166, 1082185]","[959926, 12130522, 1062002, 1085604, 844462]","[1603274, 10182720, 1088433, 1446806, 148378]","[850990, 12263035, 6533889, 916122, 6534178]","[1082185, 981760, 995242, 1029743]","[1082185, 981760, 995242, 1029743]","[1082185, 981760, 995242, 1029743]",[],"[1082185, 1029743, 995242, 6534178]"
1934,2364,"[1762083, 1765104, 1776967, 1784407, 1786062, ...","[7409974, 8090942, 5995704, 1223225, 1128161]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 972558, 1137867, 951590, 8249327]","[80251, 13003226, 130507, 1090432, 1364616]","[922651, 1028891, 6534178, 1043751, 9242918]","[1082185, 981760, 995242, 1029743]","[1082185, 981760, 995242, 1029743]","[1082185, 981760, 995242, 1029743]",[],"[1082185, 1029743, 995242, 6534178]"


Возможно, это из-за того, что их покупки не попали в топ5000:

In [37]:
# проверка, есть ли у юзеров из zero_pred покупки из топ5000:
data_train.loc[data_train['user_id'].isin(zero_pred), 'item_id'].isin(top_5000).any()

False

Метрику для них будем считать равной нулю

In [38]:
def calc_metric(metric_func, source: pd.DataFrame):
    """ Подсчет метрики
    :param metric_func - функция измерения метрики. Первый аргумент - рекомендации, второй - актуальные значения
    :param source - данные для подсчета метрики
    """
    def metric_wrapper(pred, act):
        return metric_func(pred, act) if len(pred) != 0 else 0

    metric = pd.DataFrame()
    for col in source.columns:
        if col == 'user_id':
            metric[col] = source[col]
        elif col == 'actual':
            continue
        else:
            metric[col] = source[[col, 'actual']].apply(lambda row: metric_wrapper(*row.values), axis=1)
    return metric

Также, поскольку мы фильтруем предсказания, возможна ситуация, когда количество предсказанных элементов меньше, чем @K в метрике. В таком случае можно пойти двумя путями:
1) недостающие предсказания заполнить фиктивным значением, которое интерпретировалось бы метрикой как неверное предсказание
2) изменять @K метрики для конкретного объекта по количеству имеющихся предсказаний, но не более, чем изначально требуемое значение

В первом случае метрика будет чуть ниже, а во втором - чуть выше.

In [41]:
%%time
pr_at_k = calc_metric(partial(precision_at_k, k=5), result)
ap_at_k = calc_metric(lambda pred, act: ap_k(pred, act, k=min(5, len(pred))), result)
ndcg_at_k = calc_metric(lambda pred, act: ndcg(pred, act, k=min(5, len(pred), len(act))), result)

CPU times: user 3.27 s, sys: 0 ns, total: 3.27 s
Wall time: 3.27 s


In [44]:
summary = pd.DataFrame([pr_at_k.mean(), ap_at_k.mean(), ndcg_at_k.mean()], index=['precision@k', 'map@k', 'NDCG@k'])
summary.drop(columns='user_id')

,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn,cosine,tfidf,own_purchases,bm25
precision@k,0.000294,0.155240,0.023996,0.000196,0.041920,0.151028,0.155142,0.160921,0.219001,0.125857
map@k,0.000653,0.256811,0.054187,0.000735,0.098351,0.388256,0.384458,0.385391,0.418063,0.224648
NDCG@k,0.244739,0.235391,0.244054,0.244460,0.243665,0.244741,0.242937,0.243238,0.241492,0.245113


Предсказание на топ5000 получается качественнее.
Наиболее качественное предсказание в данном случае дает модель на основе индивидуальной истории покупок.